This script is used to perform lesion segmentation in fundus photographs. The lesions that can be segmented are hard exudates, soft exudates, microaneurysms and hemorrhages. The segmentation is based on a UNet, a CNN that takes an image as an input and that outputs a probability map indicating for every pixel the probability of belonging to a certain type of lesion or not.

In [ ]:
# import necessary libraries

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split

import time

print(tf.__version__)

2.3.0


In [ ]:
# define how many gpus are available and set a memmory limit
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPUs Available: ", len(gpus))
for i in range(len(gpus)):
    tf.config.experimental.set_virtual_device_configuration(gpus[i], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7900)]) 

Number of GPUs Available:  1


In [ ]:
strategy = tf.distribute.MirroredStrategy()
# the number of replicas that is created by the strategy should be equal to the number of GPU's available
print ('Number of synchronized replicas created: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of synchronized replicas created: 1


In [ ]:
# # read in train and test data in case Google DRIVE is used
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read in the train and test data for a certain lesion type

# Basepath depends on the lesion
# LesionType = 'SoftExudates'
LesionType = 'HardExudates'
# LesionType = 'Microaneurysms'
# LesionType = 'Hemorrhages'

In [ ]:
# Basepath for Google DRIVE:
Basepath = '/content/drive/My Drive/Stage_ENT_Studios/Data/IDRiD/' + LesionType + '/Arrays/'

# Basepath for Jupyter notebooks:
# Basepath = 'C:/Users/lunam/Documents/1steMaster/Stage/Data_FinalArrays/IDRiD/'+ LesionType+'/Arrays/'

# Basepath for KILI
# Basepath = '/home/kili/Desktop/Data_FinalArrays/IDRiD/'+ LesionType+'/Arrays/'

# train data
train_images = np.load(Basepath + 'train_images_Final.npy')
print('Shape train images: {}'.format(train_images.shape))

train_annotations =  np.load(Basepath + 'train_annotations_Final.npy')
# train_annotations = np.expand_dims(train_annotations, axis = 3)
print('Shape train annotations: {}'.format(train_annotations.shape))

Shape train images: (324, 256, 256, 3)
Shape train annotations: (324, 256, 256)


In [ ]:
# direction where the results of the GridSearch will be stored

# direction for Google DRIVE
dir_gridsearch = '/content/drive/My Drive/Stage_ENT_Studios/Unet/GridSearchResults/GridSearch_UNet_Sigmoid_'+ LesionType +'.txt'

# direction for jupyter notebooks
# dir_gridsearch = 'C:/Users/lunam/Documents/1steMaster/Stage/Code_Final/DR_classification/FeatureBasedClassification/Lesion_Segmentation/UNet_Sigmoid_GPU/GridSearchResults/GridSearch/' + LesionType + '/Gridsearch_UNetSigmoid.txt'

# direction for Kili
# dir_gridsearch = '/home/kili/Desktop/FeatureBasedClassification/Lesion_Segmentation/UNet_Sigmoid_GPU/GridSearchResults/GridSearch/' + LesionType + '/Gridsearch_UNetSigmoid.txt'

In [ ]:
# The UNet network
def UNet(drop_prob = 0.1, init_filters = 64):
    '''This function defines the original UNet network'''
    # initialization of the weights
    W_init = tf.initializers.GlorotUniform()
    
    # Unet network
            
    # LEFT part
    # input layer
    # input_image = tf.keras.layers.InputLayer(input_shape = (512,512,3))
    input_image = tf.keras.layers.Input(shape = (256,256,3))
        
    # Convolutional block 1
    conv2d_1 = tf.keras.layers.Conv2D(init_filters, (3, 3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(input_image)
    conv2d_2 = tf.keras.layers.Conv2D(init_filters, (3, 3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_1)
    pool_1 = tf.keras.layers.MaxPool2D((2, 2), (2, 2))(conv2d_2)
    dropout_1 = tf.keras.layers.Dropout(rate = drop_prob)(pool_1)
            
    # Convolutional block 2
    conv2d_3 = tf.keras.layers.Conv2D(2*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_1)
    conv2d_4 = tf.keras.layers.Conv2D(2*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_3)
    pool_2 = tf.keras.layers.MaxPool2D((2,2), (2, 2))(conv2d_4)
    dropout_2 = tf.keras.layers.Dropout(rate = drop_prob)(pool_2 )
            
    # Convolutional block 3
    conv2d_5 = tf.keras.layers.Conv2D(4*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_2)
    conv2d_6 = tf.keras.layers.Conv2D(4*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_5)
    pool_3 = tf.keras.layers.MaxPool2D((2,2), (2, 2))(conv2d_6)
    dropout_3 = tf.keras.layers.Dropout(rate = drop_prob)(pool_3)
            
    # Convolutional block 4
    conv2d_7 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_3)
    conv2d_8 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_7)
    pool_4 = tf.keras.layers.MaxPool2D((2,2), (2, 2))(conv2d_8)
    dropout_4 = tf.keras.layers.Dropout(rate = drop_prob)(pool_4)
            
    # MIDDLE part
    conv2d_9 = tf.keras.layers.Conv2D(16*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_4)
    conv2d_10 = tf.keras.layers.Conv2D(16*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_9)
            
            
    # RIGHT part
    # Convolutional block 1
    upsampling_1 = tf.keras.layers.UpSampling2D((2,2))(conv2d_10)
    concat_1 = tf.keras.layers.Concatenate(3)([upsampling_1, conv2d_8])
    dropout_5 = tf.keras.layers.Dropout(rate = drop_prob)(concat_1)
    conv2d_11 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_5)
    conv2d_12 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_11)
            
    # Convolutional block 2
    upsampling_2 = tf.keras.layers.UpSampling2D((2,2))(conv2d_12)
    concat_2 = tf.keras.layers.Concatenate(3)([upsampling_2, conv2d_6])
    dropout_6 = tf.keras.layers.Dropout(rate = drop_prob)(concat_2)
    conv2d_13 = tf.keras.layers.Conv2D(4*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_6)
    conv2d_14 = tf.keras.layers.Conv2D(4*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_13)
        
            
    # Convolutional block 3
    upsampling_3 = tf.keras.layers.UpSampling2D((2,2))(conv2d_14)
    concat_3 = tf.keras.layers.Concatenate(3)([upsampling_3,conv2d_4])
    dropout_7 = tf.keras.layers.Dropout(rate = drop_prob)(concat_3)
    conv2d_15 = tf.keras.layers.Conv2D(2*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_7)
    conv2d_16 = tf.keras.layers.Conv2D(2*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_15)
            
    # Convolutional block 4
    upsampling_4 = tf.keras.layers.UpSampling2D((2,2))(conv2d_16)
    concat_4 = tf.keras.layers.Concatenate(3)([upsampling_4,conv2d_2])
    dropout_8 = tf.keras.layers.Dropout(rate = drop_prob)(concat_4)
    conv2d_17 = tf.keras.layers.Conv2D(init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_8)
    conv2d_18 = tf.keras.layers.Conv2D(init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_17)
            
    # ouput layer
    output_image = tf.keras.layers.Conv2D(1, (1,1), kernel_initializer= W_init, padding = 'same')(conv2d_18)

    # define the model
    model = tf.keras.Model(inputs=input_image, outputs=output_image)
    return model

In [ ]:
# define some different losses
# loss defined as in the original UNet paper
# The energy function is computed by a pixel-wise soft-max over the final feature map combined with the cross-entropy loss function
def loss_UNet(predicted_logits, real_annotations, GlobalBatchSize):
    real_annotations = tf.expand_dims(real_annotations, axis = 3)
    # real_annotations = tf.convert_to_tensor(real_annotations)
    real_annotations = tf.cast(real_annotations, tf.float32)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels= real_annotations, logits= predicted_logits)

    # take the mean over all pixels 
    loss = tf.reduce_mean(loss, axis = (1,2))
    return tf.reduce_sum(loss) * (1. / GlobalBatchSize)

# define the focal cross entropy loss function
# loss function has to be defined in a way to avoid data inbalance
def loss_sfce(predicted_logits, real_annotations, GlobalBatchSize, alpha = 0.25, gamma = 2.0):
    '''
    This type of loss function tries to avoid data imbalance in image segmentation
    There are two parameters alpha and gamma, the default values are indicated
    gamma should always be greater than or equal to 0
    '''
    real_annotations = tf.expand_dims(real_annotations, axis = 3)
    real_annotations = tf.cast(real_annotations, tf.float32)
  
    # classic binary cross_entropy is calculated
    ce = K.binary_crossentropy(real_annotations, predicted_logits, from_logits= True)
  
    # binary cross-entropy is multiplied with two factors: alpha and modulating factor
    # convert the logits predictions into probabilities
    alpha_factor = 1.0
    modulating_factor = 1.0
    pred_prob = tf.sigmoid(predicted_logits)
  
    if alpha:
        alpha = tf.convert_to_tensor(alpha, dtype=K.floatx())
        alpha_factor = real_annotations * alpha + (1 - real_annotations) * (1 - alpha)

    p_t = (real_annotations * pred_prob) + ((1 - real_annotations) * (1 - pred_prob))
    if gamma:
        gamma = tf.convert_to_tensor(gamma, dtype=K.floatx())
        modulating_factor = tf.pow((1.0 - p_t), gamma)
  
    # compute the final loss and return
    loss = alpha_factor * modulating_factor * ce
    loss = tf.reduce_mean(loss, axis =(1,2))
    return tf.reduce_sum(loss) * (1. / GlobalBatchSize)


# Asymmetric similarity loss function, to balance recall and precision
# the larger beta, the more important the recall becomes relative to the precision
def loss_asl(predicted_logits, real_annotations, GlobalBatchSize, beta = 2):
    real_annotations = tf.cast(real_annotations, tf.float32)
    pred_prob = tf.nn.sigmoid(predicted_logits)[:,:,:,0]
  
    prod_pos = pred_prob * real_annotations
    sum_prod_pos = tf.reduce_sum(tf.reduce_sum(prod_pos, axis = 2), axis = 1)
    prod_neg_pred = (1-pred_prob) * real_annotations
    sum_prod_neg_pred = tf.reduce_sum(tf.reduce_sum(prod_neg_pred, axis = 2), axis = 1)
    prod_neg_real = (pred_prob) * (1-real_annotations)
    sum_prod_neg_real = tf.reduce_sum(tf.reduce_sum(prod_neg_real, axis = 2), axis = 1)

    beta = tf.convert_to_tensor(beta, dtype=K.floatx())

    num = (1+beta**2) * sum_prod_pos
    denom = (1+beta**2) *sum_prod_pos + beta**2 * sum_prod_neg_pred + sum_prod_neg_real

    loss = num/denom
    return tf.reduce_sum(loss) * (1. / GlobalBatchSize)

In [ ]:
def train_network(TrainImages, TrainAnnotations, TestImages, TestAnnotations, 
                  Drop_Prob = 0.1, Init_Filters = 64, batch_size = 3, loss_function = 'UNet_loss', optim = 'Adam', 
                  learning_rate = tf.Variable(1e-5, dtype=tf.float32), MAX_EPOCH = 10, SaveResults = True):
    '''
    This function trains the UNet on the indicated train data with corresponding annotations
    At the end the trained model is being saved
    '''

    # define the train and test batches that can be fed into the network
    # global batch size defines the batch size over all availabel GPU's
    print('Creating distributed data')
    Global_batch_size = batch_size * strategy.num_replicas_in_sync
    train_batch_data  = tf.data.Dataset.from_tensor_slices((TrainImages, TrainAnnotations)).shuffle(TrainImages.shape[0]).batch(Global_batch_size) 
    test_batch_data = tf.data.Dataset.from_tensor_slices((TestImages, TestAnnotations)).batch(Global_batch_size) 

    # distribute the data over the different GPU's
    train_dist_data =  strategy.experimental_distribute_dataset(train_batch_data)
    test_dist_data =  strategy.experimental_distribute_dataset(test_batch_data)

    # define the model that will be used for training and for testing
    # the model, optimisation and loss have to be distributed among GPU's
    tf.compat.v1.reset_default_graph()
    with strategy.scope():
        # model
        print('Defining the model')
        model = UNet(drop_prob = Drop_Prob, init_filters = Init_Filters)
    
        # loss
        print('Defining loss')
        def compute_loss(logits, annotations):
            if loss_function == 'UNet_loss':
                loss = loss_UNet(logits, annotations, Global_batch_size)
            elif loss_function == 'Sfce_loss':
                loss = loss_sfce(logits, annotations, Global_batch_size)
            elif loss_function == 'Asl_loss':
              loss = loss_asl(logits, annotations, Global_batch_size)
            return loss
        
        # optimization
        steps_per_epoch = int(TrainImages.shape[0]/Global_batch_size)
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate= learning_rate, decay_steps= MAX_EPOCH*steps_per_epoch*0.25, decay_rate=0.2, staircase = True)
        print('Defining optimization')
        if optim == 'Adam':
              train_op = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
        elif optim == 'sgd':
              train_op = tf.keras.optimizers.SGD(learning_rate = lr_schedule)

        # defining the metrics
        print('Defining the metrics')
        test_roc_auc = tf.keras.metrics.AUC(curve = 'ROC')
        test_pr_auc = tf.keras.metrics.AUC(curve = 'PR')
                
    # one train step is a step in which one batch of data is fed to every GPU
    def Train_Step(input):
        train_images_batch, train_annotations_batch = input

        with tf.GradientTape() as tape:
            # make prediction with model
            pred_logits = model(train_images_batch, training = True)
            # compute loss
            train_err = compute_loss(pred_logits, train_annotations_batch)
            
        # update model
        train_weights = model.trainable_variables
        gradients = tape.gradient(train_err, train_weights)
        train_op.apply_gradients(zip(gradients, train_weights))
            
    # for the last epoch some testing has to be done, in a test step one batch of test data is fed to every GPU
    def Test_Step(input):
        test_images_batch, test_annotations_batch = input
        
        # make prediction with model
        pred_logits = model(test_images_batch, training = False)
        # turn into probability map
        test_pos_prob = tf.nn.sigmoid(pred_logits)
        # compute loss
        test_err = compute_loss(pred_logits, test_annotations_batch)
        # compute auc and aupr score
        temp_test_annotations = tf.reshape(test_annotations_batch,[-1])
        temp_test_positive_prob = tf.reshape(test_pos_prob,[-1])
        test_roc_auc.update_state(temp_test_annotations, temp_test_positive_prob)
        test_pr_auc.update_state(temp_test_annotations, temp_test_positive_prob)
    
        # the error, auc and aupr score per replica are returned
        return test_err, test_roc_auc.result(), test_pr_auc.result()

    @tf.function
    def distributed_train_step(dataset_inputs):
        strategy.run(Train_Step, args=(dataset_inputs,))

    @tf.function
    def distributed_test_step(dataset_inputs):
        per_replica_losses, per_replica_roc_auc, per_replica_pr_auc = strategy.run(Test_Step, args=(dataset_inputs,))
        return per_replica_losses, per_replica_roc_auc, per_replica_pr_auc
    
    
    # the train and test steps now have to be performed with the distributed strategy
    print('Training')
    for epo in range(1,MAX_EPOCH+1):
        
        # go over all global batches
        for train_input_data in train_dist_data:
            distributed_train_step(train_input_data)

        # some testing has to be done at the last epoch
        if epo == MAX_EPOCH:
            print('Testing')
            n_test_steps = 0
            total_test_loss = 0
            total_test_auc = 0
            total_test_aupr = 0
            
            for test_input_data in test_dist_data:
                n_test_steps+=1
                per_replica_test_losses, per_replica_test_roc_auc, per_replica_test_pr_auc = distributed_test_step(test_input_data)
                total_test_loss += strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_test_losses, axis=None)
                total_test_auc += strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_test_roc_auc, axis=None)
                total_test_aupr += strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_test_pr_auc, axis=None)
                
            total_test_loss = total_test_loss/n_test_steps
            total_test_auc =  total_test_auc/n_test_steps
            total_test_aupr =  total_test_aupr/n_test_steps      

    return total_test_loss, total_test_auc, total_test_aupr   

The functions beneith can be used to perform a gridsearch over different combinations of parameters to define the parametercombination that gives the best results in terms of performance and training time.

In [ ]:
# gridsearch: look for the most optimal hyperparameters, making use of cross-validation

# for this a separate validation set has to be defined
# three fold cross-validation is performed in this case
def ThreeFoldSplit(n_split):
    '''
    This function calculates a three-fold split of the train_images and train_annotations
    Depending on n_split another train and validation set is defined
    '''
    
    a = train_images.shape[0]
    b = int(a/3)
    
    val_images = train_images[n_split*b:(n_split+1)*b]
    val_annotations = train_annotations[n_split*b:(n_split+1)*b]
    
    # split the train set in to three parts
    if n_split == 0:
        tr_images = train_images[b:]
        tr_annotations = train_annotations[b:]
        
    if n_split == 1:
        tr_images = np.vstack((train_images[0:b], train_images[2*b:]))
        tr_annotations = np.vstack((train_annotations[0:b], train_annotations[2*b:]))
        
    if n_split == 2:
        tr_images = train_images[0:2*b]
        tr_annotations = train_annotations[0:2*b]
    
    return tr_images, tr_annotations, val_images, val_annotations

In [ ]:
# function to choose the best hyperparameter combination based on model performance in terms of the ROC AUC score
def Hyperparam_Optimization(lr_list, Batch_Size_list, dropout_list, loss_func_list, optim_list, 
                            init_filters_list, epochs_list, savefile):
    n = 0
    file = open(savefile,'w') 
    start = time.time()
    for LearnRate in lr_list:
        for BatchSize in Batch_Size_list:
            for Dropout in dropout_list:
                for LossFunc in loss_func_list:
                    for Optim in optim_list:
                        for InitFilters in init_filters_list:
                            for Epoch in epochs_list:
                                
                                n +=1
                                print('Evaluating parameter combination {} ...'.format(n))
                                    
                                AUC_list = []
                                AUPR_list = []
                                Loss_list = []
                                time_list = []
                                
                                # 3-fold cross-validation is used to evaluate the model 
                                # for a certain param combination 
                                for fold in range(3):
                                        
                                    tr_images, tr_annotations, val_images, val_annotations = ThreeFoldSplit(fold)
                                    
                                    # fitting the model to the data for a certain fold and defining the auc, aupr and loss
                                    # these values can later on be compared for different param combinations
                                    start_time = time.time()
                                    
                                    val_loss, val_auc, val_aupr = train_network(tr_images, tr_annotations, val_images, val_annotations, Dropout, InitFilters, BatchSize, LossFunc, Optim, 
                                                  tf.Variable(LearnRate, dtype=tf.float32), Epoch, SaveResults = False)
                                    
                                    end_time = time.time()
                                        
                                    # creat a 1D numpy array with the predicted and true outputs   
                                    AUC_list.append(val_auc)
                                    AUPR_list.append(val_aupr)
                                    Loss_list.append(val_loss) 
                                    time_list.append(end_time-start_time) 
                                    
                                # calculate the mean and standard-dev of the AUC, AUPR, loss and time for all three folds
                                mean_AUC = np.mean(np.array(AUC_list), axis = 0)
                                std_AUC = np.std(np.array(AUC_list), axis = 0)
                                mean_AUPR = np.mean(np.array(AUPR_list), axis = 0)
                                std_AUPR = np.std(np.array(AUPR_list), axis = 0)
                                mean_Loss = np.mean(np.array(Loss_list), axis = 0)
                                std_Loss = np.std(np.array(Loss_list), axis = 0)
                                mean_time = np.mean(time_list, axis = 0)
                                    
                                # print out these values together with the training time for this parameter set
                                print('Hyperparameter combination:')
                                print('learn rate {}, batch size {}, dropout {}, loss function {}, optimization {}, initial filters {} and epochs {}'.format(LearnRate, BatchSize, Dropout, LossFunc, Optim, InitFilters, Epoch))
                                print('Results:')
                                print('AUC: mean: {}, standard deviation: {}'.format(mean_AUC, std_AUC))
                                print('AUPR: mean: {}, standard deviation: {}'.format(mean_AUPR, std_AUPR)) 
                                print('Loss: mean: {}, standard deviation: {}'.format(mean_Loss, std_Loss)) 
                                print('training time per epoch: mean: {}'.format(mean_time))

                                # save all values in a txt file
                                file.write('Hyperparameter combination: \n')
                                file.write('learn rate {}, batch size {}, dropout {}, loss function {}, optimization {}, initial filters {} and epochs {} \n'.format(LearnRate, BatchSize, Dropout, LossFunc, Optim, InitFilters, Epoch))
                                file.write('AUC: mean: {}, standard deviation: {} \n'.format(mean_AUC, std_AUC))
                                file.write('AUPR: mean: {}, standard deviation: {} \n'.format(mean_AUPR, std_AUPR))
                                file.write('Loss: mean: {}, standard deviation: {} \n'.format(mean_Loss, std_Loss))
                                file.write('training time per epoch: mean: {} \n'.format(mean_time))
                                file.write(' \n')
    print('The gridsearch took {}s'.format(time.time()- start)) 
    file.close()                                                      

In [ ]:
# # different options that have to be tried
# # learn rat
# LR_list = [1e-3, 1e-5]
# # batch size
# BS_list = [3,6]
# # drop probab
# DP_list = [0.3,0.5]
# # loss function
# LF_list = ['UNet_loss', 'Sfce_loss', 'Asl_loss']
# # optimisation function
# OF_list = ['Adam', 'sgd']
# # initial amount of filters
# IF_list = [64, 32]
# # amount of epochs
# EP_list = [200]
# # alpha and gamma, parameters of the focal loss function

# Hyperparam_Optimization(lr_list = LR_list, Batch_Size_list = BS_list, dropout_list = DP_list, 
#                         loss_func_list = LF_list, optim_list = OF_list, init_filters_list = IF_list, 
#                         epochs_list = EP_list, savefile = dir_gridsearch)

In [ ]:
# # different options that have to be tried
# # learn rat
# LR_list = [1e-3, 1e-5]
# # batch size
# BS_list = [3,6]
# # drop probab
# DP_list = [0.3,0.5]
# # loss function
# LF_list = ['UNet_loss', 'Sfce_loss', 'Asl_loss']
# # optimisation function
# OF_list = ['Adam', 'sgd']
# # initial amount of filters
# IF_list = [64, 32]
# # amount of epochs
# EP_list = [1]
# # alpha and gamma, parameters of the focal loss function

# Hyperparam_Optimization(lr_list = LR_list, Batch_Size_list = BS_list, dropout_list = DP_list, 
#                         loss_func_list = LF_list, optim_list = OF_list, init_filters_list = IF_list, 
#                         epochs_list = EP_list, savefile = dir_gridsearch)

In [ ]:
# different options that have to be tried
# learn rat
LR_list = [1e-3, 1e-5]
# batch size
BS_list = [3,6]
# drop probab
DP_list = [0.3,0.5]
# loss function
LF_list = ['UNet_loss']
# optimisation function
OF_list = ['Adam']
# initial amount of filters
IF_list = [64, 32]
# amount of epochs
EP_list = [1]
# alpha and gamma, parameters of the focal loss function

Hyperparam_Optimization(lr_list = LR_list, Batch_Size_list = BS_list, dropout_list = DP_list, 
                        loss_func_list = LF_list, optim_list = OF_list, init_filters_list = IF_list, 
                        epochs_list = EP_list, savefile = dir_gridsearch)

Evaluating parameter combination 1 ...
Creating distributed data
Defining the model
Defining loss
Defining optimization
Defining the metrics
Training
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
Testing
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/de